In [ ]:
%pip install beautifulsoup4 requests  # 서로 의존관계를 파악해서 맞는 버전으로 설치

In [3]:
# 라이브러리 불러오기
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [23]:
# 크롤링 가능여부 확인
import urllib.robotparser
rp = urllib.robotparser.RobotFileParser()
rp.set_url("https://www.hollys.co.kr/robots.txt")
rp.read()
url = 'https://www.hollys.co.kr/store/korea/korStore2.do'
rp.can_fetch('*',url)

True

In [ ]:
# 위키피디아 미국 ETF 웹 페이지에서 필요한 정보를 스크래핑하여 딕셔너리 형태로 변수 etfs에 저장
# 크롤링 정책중에 User-Agent Policy. 가 있으면  header 정보를 추가하면 가능
# 크롤링 가능여부는 robot.txt
url = "https://en.wikipedia.org/wiki/List_of_American_exchange-traded_funds"
header = {
"User-Agent": "CoolBot/0.0 (https://example.org/coolbot/; coolbot@example.org) generic-library/0.0"
}

resp = requests.get(url, headers=header )
soup = BeautifulSoup(resp.text, 'lxml')   
rows = soup.select('#mw-content-text > div.mw-content-ltr.mw-parser-output > ul:nth-child(11) > li') 
for row in rows:
    print(row.text)

iShares Core S&P Total US Stock Mkt (NYSE Arca: ITOT)
iShares MSCI ACWI Index (Nasdaq: ACWI)
iShares Russell 3000 Index (NYSE Arca: IWV)
Schwab US Broad Market ETF (NYSE Arca: SCHB)
Schwab Fundamental U.S. Broad Market Index ETF (NYSE Arca: FNDB)
Vanguard Total World Stock (NYSE Arca: VT), tracks the FTSE All-World Index
Vanguard Total Stock Market (NYSE Arca: VTI), tracks the MSCI US Broad Market Index
Vanguard Total International Stock (NYSE Arca: VXUS), tracks the MSCI All Country World ex-USA Investable Market Index
Vanguard Russell 3000 (NYSE Arca: VTHR), tracks 98% of the US market


In [3]:
etfs = {}
for row in rows:
    
    try:
        etf_name = re.findall("(.+?)\s\(", row.text)
        etf_market = re.findall("\((.+?):", row.text)
        etf_ticker = re.findall(":\s(.+)\)", row.text)
        
        if (len(etf_ticker) > 0) & (len(etf_market) > 0) & (len(etf_name) > 0):
            etfs[etf_ticker[0]] = [etf_market[0], etf_name[0]]

    except AttributeError as err:
        pass    

# etfs 딕셔너리 출력
print(etfs)

{'ITOT': ['NYSE\xa0Arca', 'iShares Core S&P Total US Stock Mkt'], 'ACWI': ['Nasdaq', 'iShares MSCI ACWI Index'], 'IWV': ['NYSE\xa0Arca', 'iShares Russell 3000 Index'], 'SCHB': ['NYSE\xa0Arca', 'Schwab US Broad Market ETF'], 'FNDB': ['NYSE\xa0Arca', 'Schwab Fundamental U.S. Broad Market Index ETF'], 'VT': ['NYSE\xa0Arca', 'Vanguard Total World Stock'], 'VTI': ['NYSE\xa0Arca', 'Vanguard Total Stock Market'], 'VXUS': ['NYSE\xa0Arca', 'Vanguard Total International Stock'], 'VTHR': ['NYSE\xa0Arca', 'Vanguard Russell 3000']}


In [4]:
# etfs 딕셔너리를 데이터프레임으로 변환
df = pd.DataFrame(etfs)
df

,ITOT,ACWI,IWV,SCHB,FNDB,VT,VTI,VXUS,VTHR
0,NYSE Arca,Nasdaq,NYSE Arca,NYSE Arca,NYSE Arca,NYSE Arca,NYSE Arca,NYSE Arca,NYSE Arca
1,iShares Core S&P Total US Stock Mkt,iShares MSCI ACWI Index,iShares Russell 3000 Index,Schwab US Broad Market ETF,Schwab Fundamental U.S. Broad Market Index ETF,Vanguard Total World Stock,Vanguard Total Stock Market,Vanguard Total International Stock,Vanguard Russell 3000
